In [1]:
sy, sx = (6, 7)
s = np.zeros(shape=(sy, sx), dtype=np.int8)
s[5, 0] = 1
s[4, 0] = -1
# print(s)

def save_svg(name, buf: np.ndarray, figsize=(1.5, 2)):
    
    sy, sx = buf.shape
    
    fig = plt.figure(figsize=figsize)
    ax = fig.gca()
    # ax = plt.gca()
    ax.set_xlim(auto=True)
    ax.set_ylim(auto=True)
    ax.set_yticks(ticks=[])
    ax.set_xticks(ticks=np.arange(0, sx))
    plt.vlines(x=np.arange(0, sx)+0.5, ymin=sy-1+0.5, ymax=-0.5, color="black")
    plt.hlines(y=np.arange(0, sy)+0.5, xmin=sx-1+0.5, xmax=-0.5, color="black")
    im = plt.imshow(buf, cmap=connect_cmap) 
    plt.savefig(name, format='svg')
    plt.close() # don't show image

save_svg('toto.svg', s)

hashlib.md5(s).hexdigest()

NameError: name 'np' is not defined

In [ ]:
import numpy as np
import sys
import traceback
from IPython.display import clear_output
from itertools import groupby
from time import sleep
import random as rd
import time
import hashlib
import graphviz
from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap

class Colors:
    RESET = "\x1b[0m"
    RED = '\033[91m'
    YELLOW = '\033[93m'

class DumpTool:
    
    colors = ['red', 'white', 'yellow']
    connect_cmap = ListedColormap(colors)
    
    @staticmethod
    def save_svg(fname: str, format: str, buf: np.ndarray, figsize:tuple=(1.5, 2)):

        sy, sx = buf.shape

        fig = plt.figure(figsize=figsize)
        ax = fig.gca()
        # ax = plt.gca()
        ax.set_xlim(auto=True)
        ax.set_ylim(auto=True)
        ax.set_yticks(ticks=[])
        ax.set_xticks(ticks=np.arange(0, sx))
        plt.vlines(x=np.arange(0, sx)+0.5, ymin=sy-1+0.5, ymax=-0.5, color="black")
        plt.hlines(y=np.arange(0, sy)+0.5, xmin=sx-1+0.5, xmax=-0.5, color="black")
        im = plt.imshow(buf, cmap=DumpTool.connect_cmap) 
        plt.savefig('.'.join([fname, format]), format=format)
        plt.close() # don't show image
    
    
class ConnectN:

    HUMAN, AI = 1, -1

    def __init__(self, connect_n=4, sx=7, sy=6, max_depth=3):

        assert (
            sx >= connect_n and sy >= connect_n
        ), f"invalid dimensions / n-connect number: ({sx}, {sy}) => ({connect_n}) ?!"

        self.sx, self.sy, self.connect_n, self.max_depth = sx, sy, connect_n, max_depth
        
        assert max_depth >= 1, 'minimum depth is 1'
        print(f'connect:{self.connect_n}; minimax depth:{self.max_depth}')

    def get_new_board(self) -> np.ndarray:
        return np.zeros(shape=(self.sy, self.sx), dtype=int)

    @staticmethod
    def dump_digraph(boards_record, fname):
        digraph = graphviz.Digraph(format='png')
        for hsh, node in boards_record.items():
            score, depth = node['score'], node['depth']
            DumpTool.save_svg(fname=hsh, format='png', buf=node['board'])
            digraph.node(hsh, label=f'{hsh[:8]} score={score} depth={depth}', image=f'{hsh}.png', labelloc='t')
            
            for c in node['children']:
                child_hash, score, board = c['hash'], c['score'], c['board']
                
                DumpTool.save_svg(fname=child_hash, format='png', buf=board)
                digraph.node(child_hash, f'{child_hash[:8]} score={score}', image=f'{child_hash}.png', labelloc='t')
                digraph.edge(hsh, child_hash)
                
        digraph.render()
        
    @staticmethod
    def print(c4, cinfo = ()):

        sy, sx = c4.shape
        symbols = {0: " . ", 1: " o ", -1: " x "}

        header = [f"{x:02d}." for x in range(sx)]
        print(*header)
        for i in range(sy):
            L = []
            for j in range(sx):
                v = symbols[c4[i, j]]
                L.append(f"{v:3}")
                
            print(*L, flush=True)
            
    @staticmethod
    def hash_board(board: np.ndarray):
        return hashlib.md5(board).hexdigest()
        

    @staticmethod
    def is_valid_column(board, col):
        _, sx = board.shape
        return col >= 0 and col < sx and 0 in board[:, col]

    @staticmethod
    def play(board, j, is_max_player):

        sy, sx = board.shape
        if j and not ConnectN.is_valid_column(board, j):
            raise RuntimeError(f"invalid column {j}", file=sys.stderr)

        bc = board.copy()  # don't modify the input

        def drop(j, is_max_player):
            for i in range(sy - 1, -1, -1):
                if bc[i, j] == 0:
                    bc[i, j] = ConnectN.AI if is_max_player else ConnectN.HUMAN
                    return True
            return False

        if j is not None:
            st = drop(j, is_max_player)
            assert st
        else:
            for j in rd.sample(range(sx), sx):
                st = drop(j, is_max_player)
                if st:
                    break
        return bc

    def run(self):

        is_max_player = False  # e.g. is AI playing?
        is_game_over = False
        c4 = self.get_new_board()
        in_error = False
        
        while not in_error:
            try:
                ConnectN.print(c4)
                if is_game_over:
                    break

                clear_output(wait=True) #(not is_max_player))
                if is_max_player:
                    solution = {}
                    t0 = time.process_time()
                    # score = self.negamax(c4, self.max_depth, -np.inf, +np.inf, 1, solution)
                    boards_record = {}
                    score = self.minimax(c4, self.max_depth, alpha=-np.inf, beta=+np.inf, is_max_player=True, solution=solution, 
                                         boards_record=boards_record)
                    j = solution.get('col')
                    print(f"[AI] {solution} (time:{round(time.process_time() - t0, 3)}s)")
                    
                    # update the topmost board witht the best solution found (AI)
                    dump_boards = boards_record.copy()
                    for k, v in boards_record.items():
                        if v['depth'] == self.max_depth:
                            dump_boards[k]['board'] = self.play(v['board'], j, True)
                            break
                    self.dump_digraph(dump_boards, 'test.png')
                else:
                    j = int(input("play:"))

                c4 = ConnectN.play(c4, j, is_max_player)

                winfo = ConnectN.is_winning_move(c4, self.connect_n, is_max_player)
                if winfo:
                    is_game_over = True
                    pyr = "AI" if is_max_player else "PLAYER"
                    print(f"*** Game Over! Winner: {pyr} ***")
                    ConnectN.print(c4, winfo)
                    break

                is_max_player = not is_max_player
                in_error = False
            except Exception as e:
                print(e, file=sys.stderr)
                in_error = True

    @staticmethod
    def line_count_max_connect(line, val):
        max_connect = 0
        for l in [list(v) for _, v in groupby(line)]:
            if np.equal(l, val).all():
                max_connect = max(max_connect, len(l))
        return max_connect

    @staticmethod
    def is_winning_move(board, connect_n, is_max_player):

        sy, sx = board.shape
        assert (
            connect_n <= sx and connect_n <= sy
        ), f"invalid dimensions / n-connect number: ({sx}, {sy}) => ({connect_n}) ?!"

        val = ConnectN.AI if is_max_player else ConnectN.HUMAN

        # check for a horizontal win - count consecutive 1s
        for i in range(sy):
            line = board[i, :]
            if not any(line):
                continue
            if ConnectN.line_count_max_connect(line, val) >= connect_n:
                return (i, 'H')

        # check for a vertical win
        for j in range(sx):
            line = board[:, j]
            if not any(line):
                continue
            if ConnectN.line_count_max_connect(line, val) >= connect_n:
                return (j, 'V')

        # check for a diagonal win
        for offset in range(-(sy - 1), sx):
            line = np.diagonal(board, offset)
            if len(line) < connect_n or not any(line):
                continue
            if ConnectN.line_count_max_connect(line, val) >= connect_n:
                return (offset, 'D')

        # check for an anti-diagonal win
        for offset in range(-(sy - 1), sx):
            line = np.diagonal(np.flipud(board), offset)  # vertical flip
            if len(line) < connect_n or not any(line):
                continue
            if ConnectN.line_count_max_connect(line, val) >= connect_n:
                return (offset, 'AD')

        return None

    @staticmethod
    def is_last_move(node, connect_n):
        nb_turn_left = len(list(filter(lambda x: x == 0, node.flatten())))
        return True if nb_turn_left == 1 else False

    @staticmethod
    def score(node: np.array, connect_n: int, negamax_color: int = 1) -> float:
        ''' give a high value for a board if maximizer‘s turn or a low value for the board if minimizer‘s turn '''

        sy, sx = node.shape
        score = 0

        pyr, opp = (ConnectN.AI, ConnectN.HUMAN) if negamax_color > 0 else (ConnectN.HUMAN, ConnectN.AI)

        def get_updated_score(line, score):
            """ Compute score based on heuristic function """
            nb_connect_pyr = line.tolist().count(pyr)
            nb_connect_opp = line.tolist().count(opp)
            #nb_connect_empty = line.tolist().count(0)
            
            if nb_connect_pyr >= 1 and not nb_connect_opp:  # only empty left
                score += nb_connect_pyr #int(nb_connect_pyr ** 2)
            elif nb_connect_opp >= 1 and not nb_connect_pyr:
                score -= nb_connect_opp # int(nb_connect_opp ** 2)
            
            return score

        # check for a horizontal win
        for i in range(sy):

            line = node[i, :]

            if not any(line):
                continue

            for i in range(0, len(line) - connect_n + 1):
                sub_line = line[i:i+connect_n]
                score = get_updated_score(sub_line, score)

        #print(pyr, 'depth', depth, 'pyr:', nb_connect_pyr, 'opp:', nb_connect_opp, 'empty:', nb_connect_empty, 'score:', score, 'line:', line)
        # check for a vertical win
        for j in range(sx):

            line = node[:, j]

            if not any(line):
                continue

            for i in range(0, len(line) - connect_n + 1):
                sub_line = line[i:i+connect_n]
                score = get_updated_score(sub_line, score)

        # check for a diagonal win
        for offset in range(-(sy - 1), sx):

            line = np.diagonal(node, offset)

            if len(line) < connect_n or not any(line):
                continue

            for i in range(0, len(line) - connect_n + 1):
                sub_line = line[i:i+connect_n]
                score = get_updated_score(sub_line, score)

        # check for an anti-diagonal win
        for offset in range(-(sy - 1), sx):

            line = np.diagonal(np.flipud(node), offset)  # vertical flip

            if len(line) < connect_n or not any(line):
                continue

            for i in range(0, len(line) - connect_n + 1):
                sub_line = line[i:i+connect_n]
                score = get_updated_score(sub_line, score)

        return score

    def minimax(self, board: np.ndarray, depth: int, alpha: float, beta: float, is_max_player: bool, solution,
                boards_record: dict) -> float:

        assert alpha < beta
        
        sy, sx = board.shape
            
        # When the depth limit of the search is exceeded,
        # score the node as if it were a leaf
        if depth == 0 or ConnectN.is_last_move(board, self.connect_n):
            return ConnectN.score(board, self.connect_n)
               
        root_board_hash = self.hash_board(board)
        boards_record[root_board_hash] = {'board': board, 'children': [], 'score': None, 'depth': depth}
        
        if is_max_player:
            value = -np.inf
            for k in range(0, sx):
                if 0 in board[:, k]:
                    solution['nb_node_explore'] = solution.get('nb_node_explore', 0) + 1
                    b = ConnectN.play(board, k, True)
                    
                    value_new = self.minimax(b, depth - 1, alpha, beta, False, solution, boards_record)
                    
                    boards_record[root_board_hash]['children'].append({
                      'board': b,
                      'hash': self.hash_board(b),
                      'score': value_new
                    })
                    
                    if value_new > value:  # maximize value
                        value = value_new
                        solution.update({'col': k, 'depth': depth, 'score': value,'is_max_player': is_max_player})

                    if value >= beta:  # beta pruning
                        solution['nb_beta_prune'] = solution.get('nb_beta_prune', 0) + 1
                        break

                    alpha = max(alpha, value)  # no fail-soft

        else:
            value = np.inf
            for k in range(0, sx):
                if 0 in board[:, k]:
                    solution['nb_node_explore'] = solution.get('nb_node_explore', 0) + 1
                    b = ConnectN.play(board, k, False)
                    
                    value_new = self.minimax(b, depth - 1, alpha, beta, True, solution, boards_record)
                    
                    boards_record[root_board_hash]['children'].append({
                      'board': b,
                      'hash': self.hash_board(b),
                      'score': value_new
                    })
                    
                    if value_new < value:  # minimize value
                        value = value_new
                        #solution.update({'col': k, 'depth': depth, 'score': value,
                        #                 'is_max_player': is_max_player, 'hash': self.hash_board()})

                    if value <= alpha:  # alpha pruning
                        solution['nb_alpha_prune'] = solution.get('nb_alpha_prune', 0) + 1
                        break

                    beta = min(beta, value)  # no fail-soft 

        boards_record[root_board_hash]['score'] = value
        return value
    
    def negamax(self, board: np.ndarray, depth: int, alpha: float, beta: float, color: int, solution: dict) -> float:
        
        sy, sx = board.shape
        
        # When the depth limit of the search is exceeded,
        # score the node as if it were a leaf
        if depth == 0 or ConnectN.is_last_move(board, self.connect_n):
            score = ConnectN.score(board, self.connect_n, color)
            return score * color
        
        score = -np.inf
        for k in range(0, sx):
            if 0 in board[:, k]:
                solution['nb_node_explore'] = solution.get('nb_node_explore', 0) + 1
                b = ConnectN.play(board, k, True if color > 0 else False)
                new_score = -self.negamax(b, depth-1, -beta, -alpha, -color, solution)

                if new_score > score:  
                    score = new_score
                    if depth == self.max_depth:
                        solution.update({'col': k, 'score': score, 'color': color})
                    
                alpha = max(alpha, score)
                if alpha >= beta:
                    solution['nb_prune'] = solution.get('nb_prune', 0) + 1
                    break  # cut-off
                    
        return score

connect_four = ConnectN(max_depth=2)
connect_four.run()

[AI] {'nb_node_explore': 42, 'col': 3, 'depth': 2, 'score': 8, 'is_max_player': True, 'nb_alpha_prune': 4} (time:0.006s)
00. 01. 02. 03. 04. 05. 06.
 .   .   .   .   .   .   . 
 .   .   .   .   .   .   . 
 .   .   .   x   .   .   . 
 .   .   .   x   .   .   . 
 .   .   .   x   .   .   . 
 o   x   .   o   o   o   . 
